In [231]:
import os 
import pandas as pd

# load in the csv files as dataframes
items_df = pd.read_csv(os.path.expanduser("~/Fluid-Solutions-ML/data/raw/d_items.csv"))
chart_df = pd.read_csv(os.path.expanduser("~/Fluid-Solutions-ML/data/raw/chartevents.csv"))
fluid_df = pd.read_csv(os.path.expanduser("~/Fluid-Solutions-ML/data/raw/inputevents.csv"))

In [ ]:
# extract the specific vitals/items we want from the items dataframe
features = ['central venous pressure', 'mean arterial pressure', 'spo2', 'ppv', 'blood pressure', 'heart rate', 'dialysis']

# apply a mask on the items df that looks at each row of the dataframe and finds something that matches the features
mask = items_df.apply(lambda row: any(feature in str(value).lower() for value in row for feature in features), axis=1)
filtered_items_df = items_df[mask].drop_duplicates(subset=['itemid'])

# filtered_items_df

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
96,227539,ART Blood Pressure Alarm Source,ART BP Alarm Source,chartevents,Alarms,NaN,Text,NaN,NaN
131,225976,Replacement Fluid,Replacement Fluid,chartevents,Dialysis,NaN,Text,NaN,NaN
132,225958,Heparin Concentration (units/mL),Heparin Concentration (units/mL),chartevents,Dialysis,NaN,Text,NaN,NaN
133,227290,CRRT mode,CRRT mode,chartevents,Dialysis,NaN,Text,NaN,NaN
134,224139,Dialysis Site Appearance,Dialysis Site Appearance,chartevents,Dialysis,NaN,Text,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3888,225321,Dialysis Catheter Dressing Change,Dialysis Catheter Dressing Change,datetimeevents,Access Lines - Invasive,NaN,Date and time,NaN,NaN
3897,225319,Dialysis Catheter Change over Wire Date,Dialysis Catheter Change over Wire Date,datetimeevents,Access Lines - Invasive,NaN,Date and time,NaN,NaN
3900,225324,Dialysis CatheterTubing Change,Dialysis Catheter Tubing Change,datetimeevents,Access Lines - Invasive,NaN,Date and time,NaN,NaN
3918,225322,Dialysis Catheter Insertion Date,Dialysis Catheter Insertion Date,datetimeevents,Access Lines - Invasive,NaN,Date and time,NaN,NaN


In [233]:
# get the ids for each of the items in the filtered dataframe
filtered_items_ids = list(filtered_items_df['itemid'])

In [234]:
# get the chart events (vitals that were taken) for each of the items that we found above. 
# The items found above represent the vitals itself and the chart events represent the actual data that was taken
# also remove any items that are NaN
vitals_events = chart_df[
    (chart_df['itemid'].isin(filtered_items_ids)) &
    (~chart_df['valuenum'].isna())
]

# vitals_events

In [235]:
# get a list of fluid input events (specified by the ml) that have a value != NaN 
fluid_events = fluid_df[
    (~fluid_df['amount'].isna()) & 
    (fluid_df['amountuom'].astype(str) == "ml")
]

fluid_events

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,...,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
8,10005817,20626031,32604416,6770,2132-12-16 06:23:00,2132-12-17 00:37:00,2132-12-17 00:16:00,220949,91.233010,ml,...,Main order parameter,Continuous IV,91.0,250.0,ml,0,0,FinishedRunning,91.233002,5.003638
9,10005817,20626031,32604416,92805,2132-12-15 18:58:00,2132-12-15 19:12:00,2132-12-15 18:56:00,220949,9.543986,ml,...,Mixed solution,Continuous Med,91.0,250.0,ml,0,0,ChangeDose/Rate,214.739685,40.946400
10,10005817,20626031,32604416,20310,2132-12-17 09:15:00,2132-12-17 09:16:00,2132-12-17 09:28:00,220949,500.000000,ml,...,Mixed solution,Drug Push,91.0,500.0,ml,0,0,FinishedRunning,500.000000,0.000000
11,10005817,20626031,32604416,92805,2132-12-15 16:07:00,2132-12-15 16:20:00,2132-12-15 16:30:00,220949,4.437870,ml,...,Mixed solution,Continuous Med,91.0,250.0,ml,0,0,ChangeDose/Rate,248.520706,20.473200
12,10005817,20626031,32604416,12929,2132-12-15 14:34:00,2132-12-16 06:23:00,2132-12-15 14:34:00,220949,158.166671,ml,...,Main order parameter,Continuous IV,91.0,250.0,ml,0,0,ChangeDose/Rate,250.000000,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20394,10019003,29279905,34107647,68979,2153-03-28 21:11:00,2153-03-28 21:58:00,2153-03-28 23:22:00,227522,50.000000,ml,...,Main order parameter,Continuous IV,96.0,50.0,ml,0,0,FinishedRunning,50.000000,63.829784
20395,10019003,29279905,34107647,60023,2153-03-28 11:00:00,2153-03-28 12:00:00,2153-03-28 12:21:00,227522,49.999999,ml,...,Main order parameter,Continuous IV,96.0,50.0,ml,0,0,FinishedRunning,50.000000,50.000000
20396,10019003,29279905,34107647,83144,2153-03-30 06:38:00,2153-03-30 07:38:00,2153-03-30 06:38:00,227523,49.999999,ml,...,Main order parameter,Continuous IV,96.0,50.0,ml,0,0,FinishedRunning,50.000000,50.000000
20397,10019003,29279905,34107647,8034,2153-03-31 05:35:00,2153-03-31 06:35:00,2153-03-31 06:03:00,227523,49.999999,ml,...,Main order parameter,Continuous IV,96.0,50.0,ml,0,0,FinishedRunning,50.000000,50.000000


In [236]:
# get a list of unique subject ids (patients) that have received fluid administration
unique_patients_with_fluid_admin = fluid_events['subject_id'].unique()

# unique_patients_with_fluid_admin

In [237]:
# get a list of patients who have received fluids AND are listed in the chart events
patients = vitals_events[
    vitals_events['subject_id'].isin(list(unique_patients_with_fluid_admin))
    ]
patients = patients['subject_id'].unique()

# NOTE: this list is the same as the one above, but it is just safer to do this
# patients

In [238]:
# get a list of the vitals/chart events only involving the specified patients
unique_patients_vitals_events = vitals_events[
    (vitals_events['subject_id'].isin(list(patients)))
]

unique_patients_vitals_events

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
1,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:43:00,223769,100,100.0,%,0.0
11,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:41:00,220047,55,55.0,bpm,0.0
12,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:42:00,220073,0,0.0,mmHg,0.0
25,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:42:00,220072,20,20.0,mmHg,0.0
36,10005817,20626031,32604416,6770.0,2132-12-16 00:00:00,2132-12-15 23:41:00,220046,120,120.0,bpm,0.0
...,...,...,...,...,...,...,...,...,...,...,...
668739,10019003,29279905,34107647,68979.0,2153-03-28 02:57:00,2153-03-28 02:57:00,220181,77,77.0,mmHg,0.0
668741,10019003,29279905,34107647,68979.0,2153-03-28 02:57:00,2153-03-28 02:57:00,220045,99,99.0,bpm,0.0
668743,10019003,29279905,34107647,68979.0,2153-03-28 02:57:00,2153-03-28 02:57:00,220277,92,92.0,%,0.0
668744,10019003,29279905,34107647,68979.0,2153-03-28 02:57:00,2153-03-28 02:57:00,220180,62,62.0,mmHg,0.0


Before we do this step, we have to merge/condense/clump the vitals information from the chart df down into pairs that represent what we will train the data with. To do this we either need to clump the item ids into groups of specific vitals or choose specific item ids to query off of. 

In [239]:
# Convert datetime safely using assign() to avoid modifying a slice
unique_patients_vitals_events = unique_patients_vitals_events.assign(
    storetime=pd.to_datetime(unique_patients_vitals_events['storetime'])
)
fluid_events = fluid_events.assign(
    starttime=pd.to_datetime(fluid_events['starttime'])
)

# Sort and merge
vitals_events_sorted_by_time = unique_patients_vitals_events.sort_values('storetime')
fluid_events_sorted_by_time = fluid_events.sort_values('starttime')

df_merged = pd.merge_asof(
    vitals_events_sorted_by_time, 
    fluid_events_sorted_by_time, 
    left_on='storetime', 
    right_on='starttime'
)

# df_merged

columns = df_merged.columns
columns

Index(['subject_id_x', 'hadm_id_x', 'stay_id_x', 'caregiver_id_x', 'charttime',
       'storetime_x', 'itemid_x', 'value', 'valuenum', 'valueuom', 'warning',
       'subject_id_y', 'hadm_id_y', 'stay_id_y', 'caregiver_id_y', 'starttime',
       'endtime', 'storetime_y', 'itemid_y', 'amount', 'amountuom', 'rate',
       'rateuom', 'orderid', 'linkorderid', 'ordercategoryname',
       'secondaryordercategoryname', 'ordercomponenttypedescription',
       'ordercategorydescription', 'patientweight', 'totalamount',
       'totalamountuom', 'isopenbag', 'continueinnextdept',
       'statusdescription', 'originalamount', 'originalrate'],
      dtype='object')